In [45]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [46]:
import sklearn
from sklearn.model_selection import train_test_split

In [47]:
#Input training data and split into testing and training datasets
traindata=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
xtrain,xtest,ytrain,ytest=train_test_split(traindata.drop(columns='label'),traindata['label'],test_size=0.2,random_state=4)
xtrain=xtrain.astype('float64')/255       #Normalize testing and training set pixel values
xtest=xtest.astype('float64')/255
ytrain=tf.keras.utils.to_categorical(ytrain)   #One hot encoding for number labels
ytest=tf.keras.utils.to_categorical(ytest)

In [66]:
#Build the model
from tensorflow.keras import layers
model=keras.models.Sequential()
model.add(Dense(128,activation='relu',input_shape=(784,)))
# model.add(Dropout(0.10))
model.add(Dense(128,activation='relu'))
# model.add(Dropout(0.10))
model.add(Dense(128,activation='relu'))
model.add(Dense(10,activation='softmax'))

#Compile and train
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(xtrain,ytrain,epochs=10,batch_size=4200)

Epoch 1/10
8/8 [==============================] - 1s 34ms/step - loss: 2.0071 - accuracy: 0.4443
Epoch 2/10
8/8 [==============================] - 0s 35ms/step - loss: 1.1476 - accuracy: 0.7624
Epoch 3/10
8/8 [==============================] - 0s 35ms/step - loss: 0.5417 - accuracy: 0.8551
Epoch 4/10
8/8 [==============================] - 0s 34ms/step - loss: 0.3747 - accuracy: 0.8903
Epoch 5/10
8/8 [==============================] - 0s 35ms/step - loss: 0.3123 - accuracy: 0.9093
Epoch 6/10
8/8 [==============================] - 0s 34ms/step - loss: 0.2704 - accuracy: 0.9215
Epoch 7/10
8/8 [==============================] - 0s 44ms/step - loss: 0.2399 - accuracy: 0.9304
Epoch 8/10
8/8 [==============================] - 0s 36ms/step - loss: 0.2176 - accuracy: 0.9357
Epoch 9/10
8/8 [==============================] - 0s 35ms/step - loss: 0.2012 - accuracy: 0.9409
Epoch 10/10
8/8 [==============================] - 0s 33ms/step - loss: 0.1864 - accuracy: 0.9457


In [67]:
#Loss and accuracy without dropout layer   16 neurons 0.6658 0.8399
#                                          24 neurons 0.5003 0.8681
#                                          128 neurons 0.2522 0.9308
#                                          1024 neurons 0.1144 0.9660

score=model.evaluate(xtest,ytest)
print('Score loss,accuracy',score)

263/263 [==============================] - 1s 2ms/step - loss: 0.2147 - accuracy: 0.9410
Score loss,accuracy [0.2147366851568222, 0.9409523606300354]


In [62]:
#Load prediction dataset
x_pred=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
x_pred=x_pred.astype('float64')/255
y_pred=model.predict(x_pred)

In [42]:
y_pred

array([[1.6799946e-06, 6.6949890e-11, 9.9998617e-01, ..., 2.3015627e-07,
        1.5087031e-06, 8.3354145e-08],
       [9.9992085e-01, 8.8455687e-09, 2.3468363e-06, ..., 5.1058629e-07,
        3.1615151e-08, 4.9380208e-08],
       [2.4557887e-05, 4.5754312e-04, 3.9618588e-03, ..., 1.3760458e-02,
        1.6005082e-02, 8.6593032e-01],
       ...,
       [6.6434729e-09, 9.2170822e-08, 1.5419523e-07, ..., 8.7978535e-08,
        5.2157234e-06, 1.3941424e-05],
       [1.6443388e-06, 4.4158037e-09, 1.3106680e-07, ..., 1.5709871e-04,
        2.5346139e-06, 9.9017161e-01],
       [6.6553116e-06, 1.2556549e-08, 9.9972552e-01, ..., 4.4526679e-08,
        9.0063419e-05, 1.7419441e-05]], dtype=float32)

In [63]:
#Writer function to write label with maximum activation
import csv
with open('/kaggle/working/output.csv','w') as f:
    writer=csv.writer(f)
    writer.writerow(['ImageID','Label'])
    j=1
    for x in y_pred:
        for i in range(10):
            if x[i]==max(x):
                writer.writerow([j,i])
                break
        j+=1
    